In [4]:
%load_ext autoreload

%autoreload 2

In [5]:
import os
import sys
import lmdb
import pickle
import numpy as np

In [6]:
os.chdir('../') # adress to git dir
print(os.getcwd())

/home/azeghost/git/lmdb_new/LMDB_Datasets


In [ ]:
# from tensorflow.keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
# from transformation.file_utils import get_file_path
# from transformation.file_image_generator import create_image_lists

In [7]:
from transformation.lmdb_transformer import LmdbTransformer

In [ ]:
# Store images in lmdb

In [ ]:
!rm -rf storage.googleapis.com

In [ ]:
!wget https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz -O /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/mnist.npz

In [ ]:
!ls -la ./.data

In [8]:
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data(path='/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/mnist.npz')

In [ ]:
class LmdbTransformer:
    def __init__(self,image_dir = None, validation_pct = 30, valid_image_formats = ['.png'],
                 data_format= None, scalar = 255.0, *args, **kwargs):
        #args -- tuple of anonymous arguments
        #kwargs -- dictionary of named arguments
#         self.image_dir = kwargs.get('image_dir',)
#         self.validation_pct = kwargs.get('validation_pct',) 
#         self.valid_image_formats = kwargs.get('valid_image_formats',) 
#         self.data_format= None = kwargs.get('data_format',)
#         self.scalar = 255.0 = kwargs.get('scalar',)
        

        if not image_dir is None:
            self.image_lists =create_image_lists(image_dir, validation_pct, valid_image_formats)
        else:
            self.images = kwargs.get('images')
            self.labels = kwargs.get('labels')
        self.scaler = scalar 
        if data_format is None:
           self.data_format = K.image_data_format()
        else:
           self.data_format = data_format

In [ ]:
import numpy as np

#Wrapper class for dataset
class DatasetWrapper:
    def __init__(self, image, labels_dict):
        try:
            self.channels = image.shape[2]
        except:
            self.channels = 1
        self.size = image.shape[:2]
        self.image = image.tobytes()
#         for k, val in labels_dict.items():
#             exec(f'self.{k}={val}')
        self.label = labels_dict #additional data to be stored (make it string)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32) #pay attention if you  don't use create_image_lists
        return images.reshape(*self.size, self.channels)     #then dtype will be different

In [ ]:
def store_single_lmdb(filename, img, index, labels_dict, num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        # All key-value pairs need to be Strings
        value = DatasetWrapper(img, labels_dict)
        key = f"{index:08}"
        txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

In [ ]:

def transform_store_from_numpy(images, labels,
               lmdb_dir='.data/', category='training', target_size=None,
               color_mode='rgb'):
    create_if_not_exist(lmdb_dir)

    classes = list(set(labels))
    num_class = len(classes)
#     class2id = dict(zip(classes, range(len(classes))))
#     id2class = dict((v, k) for k, v in class2id.items())

#     for label_name in classes:
#         filter_arr = labels == label_name
        
#         fil_images = images[filter_arr]
#         fil_labels = labels[filter_arr]
#         num_images_per_label = fil_labels.shape[0]
        
#         print('Storing ' + str(num_images_per_label) + ' of label {}'.format(str(label_name)) +
#               ' In ' +lmdb_dir + os.sep + '_{}_{}'.format(category, str(label_name))
    index = 0
    for image, label in zip(fil_images, fil_labels):
#             / self.scaler
        img = np.float32(image)
        lmdb_name = lmdb_dir + os.sep + '_{}_{}'.format(category, str(label_name))
        store_single_lmdb(index=index, filename=lmdb_name, img=img, labels_dict=label, num_images=num_images_per_label)
        index = index + 1

In [ ]:
transform_store_from_numpy(images =X_train,  labels=Y_train,
               lmdb_dir='.data/MNIST_LMDB', category='training', target_size=None,
               color_mode='rgb')

In [10]:
!mkdir .data/MNIST_LMDB

In [9]:
!rm -rf .data/MNIST_LMDB

In [11]:
!ls .data/MNIST_LMDB

In [12]:
# Transform from .py calls

In [13]:
from transformation.lmdb_transformer import LmdbTransformer

In [14]:
validation_percentage = 30
valid_format = 'png'

In [15]:
numpy_transformer = LmdbTransformer( validation_percentage, valid_format)

In [16]:
numpy_transformer.transform_store_from_numpy(images =X_test,   labels=Y_test,
               lmdb_dir='.data/MNIST_LMDB', category='training', target_size=None,
               color_mode='rgb')

  0%|          | 18/10000 [00:00<01:22, 121.01it/s]

Storing 10000.data/MNIST_LMDB _training


100%|██████████| 10000/10000 [00:41<00:00, 240.93it/s]


In [17]:
numpy_transformer.transform_store_from_numpy(images =X_train,  labels=Y_train,
               lmdb_dir='.data/MNIST_LMDB', category='validation', target_size=None,
               color_mode='rgb')

  0%|          | 41/60000 [00:00<02:26, 408.29it/s]

Storing 60000.data/MNIST_LMDB _validation


100%|██████████| 60000/60000 [02:35<00:00, 384.88it/s]


# Read the lmdb 

#Todo
 - Env put outside of loops for write and read
 - Try the Dynamic code again with clean kernel 
 - Create Standartize as an option

In [20]:
%cd .data/


/home/azeghost/git/lmdb_new/LMDB_Datasets/.data


In [22]:
!ls -la ./MNIST_LMDB

total 16
drwxr-xr-x 4 azeghost azeghost 4096 Sep  5 21:29 .
drwxrwxr-x 5 azeghost azeghost 4096 Sep  5 21:28 ..
drwxr-xr-x 2 azeghost azeghost 4096 Sep  5 21:28 _training
drwxr-xr-x 2 azeghost azeghost 4096 Sep  5 21:29 _validation


In [34]:
import pickle
def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True) # Start a new read transaction
    
    with env.begin() as txn:
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            
            
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

            for label in labels_list:
                _lab = {label: eval(f'dataset.{label}')}
                labels = {**labels, **_lab}
                
    env.close()
    return {'images': images, **labels}


In [35]:
ds = read_many_lmdb('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/MNIST_LMDB/_training', 60000)


TypeError: a bytes-like object is required, not 'NoneType'

In [ ]:
ds.keys()

In [ ]:
ds['images'][0].shape

In [36]:
!tar -cvjf mnist.tar.bz MNIST_LMDB/* 

!split -b 50M mnist.tar.bz "mnist.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

#Check if they are created
!ls -lh mnist.tar.part*

import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')

!ls -la

#move to correct folder and push to git
# !mkdir /home/azeghost/git/LMDB_Datasets/pokemon
# !mv pokemon.tar.part* /home/azeghost/git/Generative_Models/data/.pokemon
# !ls -la /home/azeghost/git/Generative_Models/data/.pokemon 

# Combine Them

#pokemon_training
!cat mnist.tar.part* > mnist_combined.tar.bz
#!cat <split files put * at the end> > <final zip name>
!ls -la 
!pwd

!mkdir ./.test

!tar -xf mnist_combined.tar.bz --directory ./.test

!ls -la ./.test/Pokemon_LMDB

# !rm -rf ./.test

MNIST_LMDB/_training/
MNIST_LMDB/_training/lock.mdb
MNIST_LMDB/_training/data.mdb
MNIST_LMDB/_validation/
MNIST_LMDB/_validation/lock.mdb
MNIST_LMDB/_validation/data.mdb
-rw-r--r-- 1 azeghost azeghost 11M Sep  5 21:56 mnist.tar.partaa
total 227772
drwxrwxr-x 5 azeghost azeghost     4096 Sep  5 21:56 .
drwxr-xr-x 8 azeghost azeghost     4096 Aug 11 15:43 ..
drwxr-xr-x 4 azeghost azeghost     4096 Sep  5 21:29 MNIST_LMDB
-rw-r--r-- 1 azeghost azeghost 11490434 Sep  5 18:30 mnist.npz
-rw-r--r-- 1 azeghost azeghost 11196828 Sep  5 21:56 mnist.tar.bz
-rw-r--r-- 1 azeghost azeghost 11196828 Sep  5 21:56 mnist.tar.partaa
-rw-r--r-- 1 azeghost azeghost 66439133 Sep  5 17:30 pokemon_combined.tar.bz
drwxr-xr-x 4 azeghost azeghost     4096 Aug  2 20:18 Pokemon_LMDB
-rw-r--r-- 1 azeghost azeghost 66439133 Sep  5 17:21 pokemon.tar.bz
-rw-r--r-- 1 azeghost azeghost 52428800 Sep  5 17:21 pokemon.tar.partaa
-rw-r--r-- 1 azeghost azeghost 14010333 Sep  5 17:21 pokemon.tar.partab
drwxr-xr-x 3 azeghost a

## LMDBImageIterator and LMDBImageGenerator

In [ ]:
os.chdir('/home/azeghost/git/Generative_Models/')

In [ ]:
os.getcwd()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class LMDBImageIterator(Iterator):


    def __init__(self,
                 num_images,
                 category,
                 lmdb_dir,
                 batch_size,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx(),
                 ):
        
        self.category = category
        self.batch_size = batch_size
        
        
        self.lmdb_dir = lmdb_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        print("Initializing Iterator " + category +" Number of images " +str(num_images))
        print(category,lmdb_dir, batch_size,shuffle,seed)
        self.env = lmdb.open(lmdb_dir, readonly=True)
        
        Iterator.__init__(self, num_images, batch_size, shuffle, seed)
        
        
    def __del__(self):
        self.env.close()
        
        
    def _get_batches_of_transformed_samples(self, index_array):
        print(index_array)
        images, labels = [], []
        
        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

        else:
                with self.env.begin() as txn:
                    for image_id in index_array:
                        data = txn.get(f"{image_id:08}".encode("ascii"))
                        dataset = pickle.loads(data)
                        images.append(dataset.get_image())
                        labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

                        for label in labels_list:
                            _lab = {label: eval(f'dataset.{label}')}
                            labels = {**labels, **_lab}
        return {'images': images, **labels}


In [ ]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message


class LMDBImageGenerator(ImageDataGenerator):
    def flow_from_lmdb_lists(self, 
                              num_images,
                              category,
                              lmdb_dir,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              shuffle =True,
                              seed=None
                              ):

        
          

        return LMDBImageIterator(
                             num_images = num_images,
                             category = category,
                             lmdb_dir = lmdb_dir,
                             batch_size  = batch_size,
                             episode_len = episode_len,
                             episode_shift =episode_shift,
                             shuffle = shuffle,
                             seed = seed)


def get_generators( val_lmdb_dir, val_num_images, tra_lmdb_dir, tra_num_images, 
                   batch_size, episode_len=None, episode_shift=None):

    train_datagen = LMDBImageGenerator()

    valid_datagen = LMDBImageGenerator()

    train_generator = train_datagen.flow_from_lmdb_lists(
        num_images = tra_num_images,
        category='training',
        lmdb_dir=tra_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_lmdb_lists(
        num_images = val_num_images,
        category='validation',
        lmdb_dir=val_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator



In [ ]:
training_generator, testing_generator = get_generators(
    val_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_validation',
    val_num_images = 218,
    tra_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 
    tra_num_images = 591,
    batch_size=100
)

In [ ]:
x, y = training_generator.next()

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((batch_size* EPIS_LEN, ) + image_size), 
                   tf.TensorShape((batch_size* EPIS_LEN, ) + image_size)
                  )
)

_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break